In [3]:
import pandas as pd
import numpy as np

### Cleaning the data

### Vizualize tha data or sumn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import cross_val_score

df = pd.read_csv('Financial Distress.csv')
df = df.drop(columns=["Company", "Time"])

X = df.drop(columns=["Financial Distress"]).dropna()
y = df["Financial Distress"].dropna()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

lasso = make_pipeline(StandardScaler(), LassoCV(cv=10, alphas=[0.05, 0.001]))
lasso.fit(X_train, y_train)

'''
coef = lasso.named_steps["lassocv"].coef_
sel_features = X.columns[(coef != 0)]
print("Selected features:", sel_features)
print(f"Best alpha: {lasso.named_steps['lassocv'].alpha_}")
'''

val_score = lasso.score(X_val, y_val)
print(f"Validation R^2: {val_score}")

Validation R^2: 0.0648737497289007


C:\Users\danit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.545372214622148, tolerance: 0.493143628203965
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\danit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13.139916638550403, tolerance: 0.4893049694736343
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\danit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:681: ConvergenceWar